# Which GPU to use

In [1]:
multiGPU = False
whichGPU = 0
 
# Select which GPU to use
if(multiGPU):
    from keras.utils.training_utils import multi_gpu_model
else:
    import os
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    # The GPU id to use, usually either "0" or "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(whichGPU)
    
# # Do other imports now...

# Load all the functions

In [2]:
%run -i 'arena.py'

Using TensorFlow backend.


# General Parameters

In [3]:
import math

# What data to use
tableBase = '4PpKk'
convertStates = False

# Interactive (just in general if one is asked for confirmations, set to False if on autopilot over night f.x.)
askForConfirmation = False

# NN parameters
filters = [16,32,32,64,128,128,128]
filterShape = [2,2,2,2,2,2,2]
batch_size = 256
optimizer = 'Adadelta'
useBatchNorm = False
num_classes = 3
input_shape = (4,8,8)

### DON'T MODIFY BELOW ###
# Generate dataset variables
fileName = tableBase + '.hdf5'
dataSetName = tableBase + '_onlyLegal'
if not convertStates: 
    dataSetName = tableBase + '_onlyLegal_fullStates'
dataSetWdlName = tableBase + '_Wdl_onlyLegal_3Values'

# Number of Pieces
nPi =  int(dataSetName[0])
nPa = nPi - 2
nWPa = math.ceil(nPa/2)

# Experiment 1
Bengio methood
3n4 no freeze

## exp 1 parameters

In [4]:
%run -i 'arena.py'

# Parameters
sourceNet = '103' # trained on 3pc from scratch
# sourceNet = '107' # trained on 4pc from scratch ######################## MODIFY
freeze = False                           ############################### MODIFY
epochs = 10
averageOver = 5 
                          ############################### MODIFY
expDescrBaseName = "Bengio 3n4 - freeze = {} - average over {} runs - {} epochs".format(str(freeze), averageOver, epochs)

saveEveryRun = True # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dit
saveTensorboardLogs = True # save logs in ./logs dir
resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 1
plotDuringTraining = False
loadWeights = False 
askForConfirmation = False

# stuff to save in bengioResults dir
resSaveFile = '3n4'
saveDir = 'bengioResults'
# resSaveFile = resSaveFile + '-{}runAverage-{}epochs'.format(averageOver, epochs)
resSaveFileFullPath = saveDir + '/' + str(resSaveFile) + '.pkl' 
resSaveFileAllRuns = resSaveFile + '-allRuns'.format(averageOver, epochs)
resSaveFileAllRunsFullPath = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl' 
resSaveFileThisRun = resSaveFile + '-thisRun'.format(averageOver, epochs)
resSaveFileThisRunFullPath = saveDir + '/' + str(resSaveFileThisRun) + '.pkl' 

### Create model and load data

In [ ]:
# prepare save file
if not os.path.exists(resSaveFileFullPath):
    print("Save file doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFile, [])
else:
    print("Save file exists...\n")
    
if not os.path.exists(resSaveFileAllRunsFullPath):
    print("Save file for all runs doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFileAllRuns, [])
else:
    print("Save file for all runs exists...\n")

if not os.path.exists(resSaveFileThisRunFullPath):
    print("Save file for this run doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFileThisRun, [])
else:
    print("Save file for this run exists...\n")

# load data
X_train, X_test, y_train, y_test = loadData(randomState = None)

# create model
model, nnStr = createModel()
layersCount = len(model.layers)

# load old results
results = load_obj(saveDir, resSaveFile)
resultsAllRuns = load_obj(saveDir, resSaveFileAllRuns)
resultsThisRun = load_obj(saveDir, resSaveFileThisRun)

# initialize variables wrt old results
startTrainingAtLayer = len(results)
print("\nStarting/restarting TL at {} transfered layers".format(startTrainingAtLayer))
# startThisRunAt = len(resultsThisRun)
# print("\nStarting/restarting TL at average {}".format(startTrainingAtLayer))

Save file doesn't exists, creating...

Save file for all runs doesn't exists, creating...

Save file for this run doesn't exists, creating...

7436088
7436088
X_train shape: (4982178, 4, 8, 8)
y_train shape: (4982178, 1)
X_test shape: (2453910, 4, 8, 8)
y_test shape: (2453910, 1)
4982178 train samples
2453910 test samples
Done loading dataset
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 7, 7)          272       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 6, 32)         928       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 5, 32)         4128      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 4, 64)         8256      
_____________________________________________________________

In [ ]:
print(results)
print(resultsThisRun)
print(resultsAllRuns)

[]
[]
[]


### Train

In [ ]:
%run -i 'arena.py'

for copyFirstNLayers in range(startTrainingAtLayer, layersCount):
    print('\n\n')
    print('==========================================================================================')
    print('=                                                                                        =')
    print('=                    Currently transfering first {} layers, out of {}                      ='.format(copyFirstNLayers, layersCount - 1))
    print('=                                                                                        =')
    print('==========================================================================================')
    print()
    
    # check if we are at the flatten layer, and skip it if so
    if copyFirstNLayers == layersCount - 1:
        copyFirstNLayers += 1
        
#     # start accumulated score from load file in case of starting from average run ≠ 0
#     accumulatedScore = np.sum(resultsThisRun)

    # Reset variables
    resultsThisRun = []
    accumulatedScore = 0
    
    # train and average results
    for a in range(averageOver):
        print('    ==================================================================================')
        print('    =                                                                                =')
        print('    =                Currently at run {}, out of {}                                   ='.format(a+1,averageOver))
        print('    =                                                                                =')
        print('    ==================================================================================')
        print()
        
        # set experement description test
        expDescr = expDescrBaseName + '__copyLayers_{}__average_{}_of_{}'.format(copyFirstNLayers, a+1, averageOver)
        
        # save current averagePosition to tmp file
        with open(saveDir + '/' + str(resSaveFile) + '_currentPosition.txt','w') as file:
            if copyFirstNLayers == layersCount:
                location = copyFirstNLayers - 1
            else:
                location = copyFirstNLayers 
            file.write('Layers Transfered: {} out of {} \nInner avg loop position: {} out of {}'.format(location, layersCount-1, a+1, averageOver))         
            
        # load Model layers
        model = loadNFirstLayers(model, sourceNet, copyFirstNLayers , freeze)

        # load data
        X_train, X_test, y_train, y_test = loadData(randomState = None)

        # Prepare save dir
        if saveEveryRun:
            resID = genNextResultsDir(model)

        # train
        fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)

        # score and save results
        score = calcScore(model)
        if saveEveryRun:
            saveTrainResults(resID, model, logDir, score, copyFirstNLayers)

        # update Return
        accumulatedScore += score[1]
        resultsThisRun.append(score[1])
        
        # save bengio checkpoint
        dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
        
        src = saveDir + '/' + str(resSaveFileThisRun) + '.pkl'
        dest = saveDir + '/checkpoints/' + str(resSaveFileThisRun) + dateTime + '.pkl'
        if os.path.exists(src):
            shutil.move(src, dest)
            
        src = saveDir + '/' + str(resSaveFileThisRun) + '.txt'
        dest = saveDir + '/checkpoints/' + str(resSaveFileThisRun) + dateTime + '.txt'
        if os.path.exists(src):
            shutil.move(src, dest)
        
        # save results 
        save_obj(saveDir, resSaveFileThisRun, resultsThisRun)
        with open(saveDir + '/' + str(resSaveFileThisRun) + '.txt','w') as file:
            file.write(str(resultsThisRun))
            
    # append averaged results for one set of layers
    results.append(accumulatedScore/averageOver)
    resultsAllRuns.append(resultsThisRun)

    # save old results to checkpoints dir
    dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
    
    src = saveDir + '/' + str(resSaveFile) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    src = saveDir + '/' + str(resSaveFile) + '.pkl'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.pkl'
    if os.path.exists(src):
        shutil.move(src, dest)

    src = saveDir + '/' + str(resSaveFileAllRuns) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFileAllRuns) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    src = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl'
    dest = saveDir + '/checkpoints/' + str(resSaveFileAllRuns) + dateTime + '.pkl'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    # save results 
    save_obj(saveDir, resSaveFile, results)
    with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
        file.write(str(results))
    
    save_obj(saveDir, resSaveFileAllRuns, resultsAllRuns)
    with open(saveDir + '/' + str(resSaveFileAllRuns) + '.txt','w') as file:
        file.write(str(resultsAllRuns))
    
print('\n Final Results: {}'.format(results))




=                                                                                        =
=                    Currently transfering first 0 layers, out of 8                      =
=                                                                                        =

    =                                                                                =
    =                Currently at run 1, out of 5                                   =
    =                                                                                =

Using first 0 layers from results Results/103/weights.hdf5, 
Loading all weights from results 103
- 1: Resetting layer <keras.layers.convolutional.Conv2D object at 0x7f455ce62978>
- 2: Resetting layer <keras.layers.convolutional.Conv2D object at 0x7f455ce62f28>
- 3: Resetting layer <keras.layers.convolutional.Conv2D object at 0x7f455ce62c18>
- 4: Resetting layer <keras.layers.convolutional.Conv2D object at 0x7f455ce2b128>
- 5: Resetting layer <keras.layers.c

In [ ]:
sys.exit()

# Experiment 0
Converge 3>4 and rnd>4

## parameters

In [ ]:
%run -i 'arena.py'

# Parameters
weightsSource = '103' # trained on 3pc from scratch
# sourceNet = '107' # trained on 4pc from scratch ######################## MODIFY
epochs = 300
averageOver = 5
                          ############################### MODIFY
expDescrBaseName = "converge 3to4 - {} averages - {} epochs".format(averageOver, epochs)

saveEveryRun = True # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dir
saveBengioCheckPoints = True
saveTensorboardLogs = True # save logs in ./logs dir
compareResultsDuringTraining = False
resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 1
plotDuringTraining = True
loadWeights = True 
loadCheckpointWeights = False
askForConfirmation = False

# stuff to save in bengioResults dir
resSaveFile = 'conv3to4'
dictFieldName = 'acc'
saveDir = 'bengioResults'
resSaveFile = resSaveFile + '-{}runAverage-{}epochs'.format(averageOver, epochs)
resSaveFileFullPath = saveDir + '/' + str(resSaveFile) + '.pkl' 
# resSaveFileAllRuns = resSaveFile + '-allRuns'.format(averageOver, epochs)
# resSaveFileAllRunsFullPath = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl' 
# resSaveFileThisRun = resSaveFile + '-thisRun'.format(averageOver, epochs)
# resSaveFileThisRunFullPath = saveDir + '/' + str(resSaveFileThisRun) + '.pkl' 

### Create model and load data

In [ ]:
# prepare save file
if not os.path.exists(resSaveFileFullPath):
    print("Save file doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFile, {dictFieldName:[]})
else:
    print("Save file exists...\n")
    
# load data
X_train, X_test, y_train, y_test = loadData(randomState = None)

# create model
# model, nnStr = createModel()

# load old results
results = load_obj(saveDir, resSaveFile)
# resultsAllRuns = load_obj(saveDir, resSaveFileAllRuns)
# resultsThisRun = load_obj(saveDir, resSaveFileThisRun)

# initialize variables wrt old results
# startTrainingAtLayer = len(results)
# print("\nStarting/restarting TL at {} transfered layers".format(startTrainingAtLayer))
# startThisRunAt = len(resultsThisRun)
startThisRunAt = len(results[dictFieldName])
print("\nStarting/restarting TL at average {}".format(startThisRunAt))

In [ ]:
print(results)

### Train

In [ ]:
%run -i 'arena.py'

# train and average results
for a in range(startThisRunAt,averageOver):
    print('    ==================================================================================')
    print('    =                                                                                =')
    print('    =                Currently at run {}, out of {}                                   ='.format(a+1,averageOver))
    print('    =                                                                                =')
    print('    ==================================================================================')
    print()

    # set experement description test
#     expDescr = expDescrBaseName + '__copyLayers_{}__average_{}_of_{}'.format(copyFirstNLayers, a+1, averageOver)
    expDescr = expDescrBaseName + '__average_{}_of_{}'.format(a+1, averageOver)

    # save current averagePosition to tmp file
    with open(saveDir + '/' + str(resSaveFile) + '_currentPosition.txt','w') as file:
        file.write('Inner avg loop position: {} out of {}'.format(a+1, averageOver))         

    # load Model layers
#     model = loadNFirstLayers(model, sourceNet, copyFirstNLayers , freeze)
    model, nnStr = createModel()

    # load data
    X_train, X_test, y_train, y_test = loadData(randomState = None)
    
    # Prepare save dir
    if saveEveryRun:
        resID = genNextResultsDir(model)

    # train
    fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)

    # calc score 
    score = calcScore(model)
    results[dictFieldName].append(score[1])
    if saveEveryRun:
        saveTrainResults(resID, model, logDir, score)

    # save checkpoint
    if saveBengioCheckPoints:
        dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())

        src = saveDir + '/' + str(resSaveFile) + '.pkl'
        dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.pkl'
        if os.path.exists(src):
            shutil.move(src, dest)

        src = saveDir + '/' + str(resSaveFile) + '.txt'
        dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
        if os.path.exists(src):
            shutil.move(src, dest)

    # save results 
    save_obj(saveDir, resSaveFile, results)
    with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
        file.write(str(results))


print('\n Final Results: {}'.format(results))
print('\n Final Results Averate: {}'.format(np.mean(np.array(results[dictFieldName]))))


In [ ]:
dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())

src = saveDir + '/' + str(resSaveFile) + '.pkl'
dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.pkl'
if os.path.exists(src):
    shutil.move(src, dest)

src = saveDir + '/' + str(resSaveFile) + '.txt'
dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
if os.path.exists(src):
    shutil.move(src, dest)

# save results 
save_obj(saveDir, resSaveFile, results)
with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
    file.write(str(results))

# Experiment 2
Bengio methood
4n4 no freeze

## exp 2 parameters

In [ ]:
%run -i 'arena.py'

# Parameters
# sourceNet = '103' # trained on 3pc from scratch
sourceNet = '107' # trained on 4pc from scratch ######################## MODIFY
freeze = False                           ############################### MODIFY
epochs = 10
averageOver = 1
                          ############################### MODIFY
expDescrBaseName = "Bengio 4n4 - freeze = {} - average over {} runs - {} epochs".format(str(freeze), averageOver, epochs)

saveEveryRun = True # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dit
saveTensorboardLogs = True # save logs in ./logs dir
resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 1
plotDuringTraining = False
loadWeights = False 
askForConfirmation = False

# stuff to save in bengioResults dir
resSaveFile = '4n4nofreeze'
saveDir = 'bengioResults'
resSaveFile = resSaveFile + '-{}runAverage-{}epochs'.format(averageOver, epochs)
resSaveFileFullPath = saveDir + '/' + str(resSaveFile) + '.pkl' 
resSaveFileAllRuns = resSaveFile + '-allRuns'.format(averageOver, epochs)
resSaveFileAllRunsFullPath = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl' 
resSaveFileThisRun = resSaveFile + '-thisRun'.format(averageOver, epochs)
resSaveFileThisRunFullPath = saveDir + '/' + str(resSaveFileThisRun) + '.pkl' 

### Create model and load data

In [ ]:
# prepare save file
if not os.path.exists(resSaveFileFullPath):
    print("Save file doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFile, [])
else:
    print("Save file exists...\n")
    
if not os.path.exists(resSaveFileAllRunsFullPath):
    print("Save file for all runs doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFileAllRuns, [])
else:
    print("Save file for all runs exists...\n")

if not os.path.exists(resSaveFileThisRunFullPath):
    print("Save file for this run doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFileThisRun, [])
else:
    print("Save file for this run exists...\n")

# load data
X_train, X_test, y_train, y_test = loadData(randomState = None)

# create model
model, nnStr = createModel()
layersCount = len(model.layers)

# load old results
results = load_obj(saveDir, resSaveFile)
resultsAllRuns = load_obj(saveDir, resSaveFileAllRuns)
resultsThisRun = load_obj(saveDir, resSaveFileThisRun)

# initialize variables wrt old results
startTrainingAtLayer = len(results)
print("\nStarting/restarting TL at {} transfered layers".format(startTrainingAtLayer))
startThisRunAt = len(resultsThisRun)
print("\nStarting/restarting TL at average {}".format(startTrainingAtLayer))

In [ ]:
print(results)
print(resultsThisRun)
print(resultsAllRuns)

### Train

In [ ]:
%run -i 'arena.py'

for copyFirstNLayers in range(startTrainingAtLayer, layersCount):
    print('\n\n')
    print('==========================================================================================')
    print('=                                                                                        =')
    print('=                    Currently transfering first {} layers, out of {}                      ='.format(copyFirstNLayers, layersCount - 1))
    print('=                                                                                        =')
    print('==========================================================================================')
    print()
    
    # check if we are at the flatten layer, and skip it if so
    if copyFirstNLayers == layersCount - 1:
        copyFirstNLayers += 1
        
    # start accumulated score from load file in case of starting from average run ≠ 0
    accumulatedScore = np.sum(resultsThisRun)
    
    # train and average results
    for a in range(startThisRunAt,averageOver):
        print('    ==================================================================================')
        print('    =                                                                                =')
        print('    =                Currently at run {}, out of {}                                   ='.format(a+1,averageOver))
        print('    =                                                                                =')
        print('    ==================================================================================')
        print()
        
        # set experement description test
        expDescr = expDescrBaseName + '__copyLayers_{}__average_{}_of_{}'.format(copyFirstNLayers, a+1, averageOver)
        
        # save current averagePosition to tmp file
        with open(saveDir + '/' + str(resSaveFile) + '_currentPosition.txt','w') as file:
            if copyFirstNLayers == layersCount:
                location = copyFirstNLayers - 1
            else:
                location = copyFirstNLayers 
            file.write('Layers Transfered: {} out of {} \nInner avg loop position: {} out of {}'.format(location, layersCount-1, a+1, averageOver))         
            
        # load Model layers
        model = loadNFirstLayers(model, sourceNet, copyFirstNLayers , freeze)

        # load data
        X_train, X_test, y_train, y_test = loadData(randomState = None)
        
        # Prepare save dir
        if saveEveryRun:
            resID = genNextResultsDir(model)

        # train
        fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)

        # score and save results
        score = calcScore(model)
        if saveEveryRun:
            saveTrainResults(resID, model, logDir, score, copyFirstNLayers)

        # update Return
        accumulatedScore += score[1]
        resultsThisRun.append(score[1])
        
        # save checkpoint
        dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
        
        src = saveDir + '/' + str(resSaveFileThisRun) + '.pkl'
        dest = saveDir + '/checkpoints/' + str(resSaveFileThisRun) + dateTime + '.pkl'
        if os.path.exists(src):
            shutil.move(src, dest)
            
        src = saveDir + '/' + str(resSaveFileThisRun) + '.txt'
        dest = saveDir + '/checkpoints/' + str(resSaveFileThisRun) + dateTime + '.txt'
        if os.path.exists(src):
            shutil.move(src, dest)
        
        # save results 
        save_obj(saveDir, resSaveFile, results)
        with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
            file.write(str(results))
            
    # append averaged results for one set of layers
    results.append(accumulatedScore/averageOver)
    resultsAllRuns.append(resultsThisRun)

    # save old results to checkpoints dir
    dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
    
    src = saveDir + '/' + str(resSaveFile) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    src = saveDir + '/' + str(resSaveFile) + '.pkl'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.pkl'
    if os.path.exists(src):
        shutil.move(src, dest)

    src = saveDir + '/' + str(resSaveFileAllRuns) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFileAllRuns) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    src = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl'
    dest = saveDir + '/checkpoints/' + str(resSaveFileAllRuns) + dateTime + '.pkl'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    # save results 
    save_obj(saveDir, resSaveFile, results)
    with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
        file.write(str(results))
    
    save_obj(saveDir, resSaveFileAllRuns, resultsAllRuns)
    with open(saveDir + '/' + str(resSaveFileAllRuns) + '.txt','w') as file:
        file.write(str(resultsAllRuns))

    # to load:
    # results = load_obj('temp','3n4.txt')
    resultsThisRun = []
print('\n Final Results: {}'.format(results))

# Experiment 3
converging 3_8_4+

### Exp 3 Paramters

In [ ]:
%run -i 'arena.py'

# Parameters
sourceNet = '103' # trained on 3pc from scratch
# sourceNet = '107' # trained on 4pc from scratch ######################## MODIFY
freeze = False                           ############################### MODIFY
epochs = 10
averageOver = 10
                          ############################### MODIFY
expDescrBaseName = "Converging 3n4plus - average over {} runs - {} epochs".format(str(freeze), averageOver, epochs)

saveEveryRun = False # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dit
saveTensorboardLogs = False # save logs in ./logs dir
resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 0.001
plotDuringTraining = False
loadWeights = False 
askForConfirmation = False

# stuff to save in bengioResults dir
resSaveFile = '3_8_4_plus_Converge'                          ############################### MODIFY
saveDir = 'bengioResults'
resSaveFile = resSaveFile + '-{}runAverage-{}epochs'.format(averageOver, epochs)
resSaveFileFullPath = saveDir + '/' + str(resSaveFile) + '.pkl' 
resSaveFileAllRuns = resSaveFile + '-allRuns'.format(averageOver, epochs)
resSaveFileAllRunsFullPath = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl' 
resSaveFileThisRun = resSaveFile + '-thisRun'.format(averageOver, epochs)
resSaveFileThisRunFullPath = saveDir + '/' + str(resSaveFileThisRun) + '.pkl' 

### Create model and load data

In [ ]:
# prepare save file
if not os.path.exists(resSaveFileFullPath):
    print("Save file doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFile, [])
else:
    print("Save file exists...\n")
    
if not os.path.exists(resSaveFileAllRunsFullPath):
    print("Save file for all runs doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFileAllRuns, [])
else:
    print("Save file for all runs exists...\n")

if not os.path.exists(resSaveFileThisRunFullPath):
    print("Save file for this run doesn't exists, creating...\n")
    save_obj(saveDir, resSaveFileThisRun, [])
else:
    print("Save file for this run exists...\n")

# load data
X_train, X_test, y_train, y_test = loadData(randomState = None)

# create model
model, nnStr = createModel()
layersCount = len(model.layers)

# load old results
results = load_obj(saveDir, resSaveFile)
resultsAllRuns = load_obj(saveDir, resSaveFileAllRuns)
resultsThisRun = load_obj(saveDir, resSaveFileThisRun)

# initialize variables wrt old results
startTrainingAtLayer = len(results)
startTrainingAtLayer = 7
print("\nStarting/restarting TL at {} transfered layers".format(startTrainingAtLayer))
startThisRunAt = len(resultsThisRun)
print("\nStarting/restarting TL at average {}".format(startTrainingAtLayer))

In [ ]:
print(results)
print(resultsThisRun)
print(resultsAllRuns)

### Train

In [ ]:
%run -i 'arena.py'

for copyFirstNLayers in range(startTrainingAtLayer, layersCount):
    print('\n\n')
    print('==========================================================================================')
    print('=                                                                                        =')
    print('=                    Currently transfering first {} layers, out of {}                      ='.format(copyFirstNLayers, layersCount - 1))
    print('=                                                                                        =')
    print('==========================================================================================')
    print()
    
    # check if we are at the flatten layer, and skip it if so
    if copyFirstNLayers == layersCount - 1:
        copyFirstNLayers += 1
        
    # train and average results
    accumulatedScore = np.sum(resultsThisRun)
#     resultsThisRun = []
    for a in range(startThisRunAt,averageOver):
        print('    ==================================================================================')
        print('    =                                                                                =')
        print('    =                Currently at run {}, out of {}                                   ='.format(a+1,averageOver))
        print('    =                                                                                =')
        print('    ==================================================================================')
        print()
        
        # set experement description test
        expDescr = expDescrBaseName + '__copyLayers_{}__average_{}_of_{}'.format(copyFirstNLayers, a+1, averageOver)
        
        # save current averagePosition to tmp file
        with open(saveDir + '/' + str(resSaveFile) + '_currentPosition.txt','w') as file:
            if copyFirstNLayers == layersCount:
                location = copyFirstNLayers - 1
            else:
                location = copyFirstNLayers 
            file.write('Layers Transfered: {} out of {} \nInner avg loop position: {} out of {}'.format(location, layersCount-1, a+1, averageOver))         
            
        # load Model layers
        model = loadNFirstLayers(model, sourceNet, copyFirstNLayers , freeze)

        # Prepare save dir
        if saveEveryRun:
            resID = genNextResultsDir(model)

        # train
        fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)

        # score and save results
        score = calcScore(model)
        if saveEveryRun:
            saveTrainResults(resID, model, logDir, score, copyFirstNLayers)

        # update Return
        accumulatedScore += score[1]
        resultsThisRun.append(score[1])
        
        
        # save checkpoint
        dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
        
        src = saveDir + '/' + str(resSaveFileThisRun) + '.pkl'
        dest = saveDir + '/checkpoints/' + str(resSaveFileThisRun) + dateTime + '.pkl'
        if os.path.exists(src):
            shutil.move(src, dest)
            
        src = saveDir + '/' + str(resSaveFileThisRun) + '.txt'
        dest = saveDir + '/checkpoints/' + str(resSaveFileThisRun) + dateTime + '.txt'
        if os.path.exists(src):
            shutil.move(src, dest)
        
        # save results 
        save_obj(saveDir, resSaveFileThisRun, resultsThisRun)
        with open(saveDir + '/' + str(resSaveFileThisRun) + '.txt','w') as file:
            file.write(str(resultsThisRun))
            
    # append averaged results for one set of layers
    results.append(accumulatedScore/averageOver)
    resultsAllRuns.append(resultsThisRun)

    # save old results to checkpoints dir
    dateTime = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime())
    
    src = saveDir + '/' + str(resSaveFile) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    src = saveDir + '/' + str(resSaveFile) + '.pkl'
    dest = saveDir + '/checkpoints/' + str(resSaveFile) + dateTime + '.pkl'
    if os.path.exists(src):
        shutil.move(src, dest)

    src = saveDir + '/' + str(resSaveFileAllRuns) + '.txt'
    dest = saveDir + '/checkpoints/' + str(resSaveFileAllRuns) + dateTime + '.txt'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    src = saveDir + '/' + str(resSaveFileAllRuns) + '.pkl'
    dest = saveDir + '/checkpoints/' + str(resSaveFileAllRuns) + dateTime + '.pkl'
    if os.path.exists(src):
        shutil.move(src, dest)
        
    # save results 
    save_obj(saveDir, resSaveFile, results)
    with open(saveDir + '/' + str(resSaveFile) + '.txt','w') as file:
        file.write(str(results))
    
    save_obj(saveDir, resSaveFileAllRuns, resultsAllRuns)
    with open(saveDir + '/' + str(resSaveFileAllRuns) + '.txt','w') as file:
        file.write(str(resultsAllRuns))

    # to load:
    # results = load_obj('temp','3n4.txt')
    resultsThisRun = []
print('\n Final Results: {}'.format(results))

In [ ]:
resultsThisRun
# save_ob('.','x',resultsThisRun)

# Experiment 0
test batch size vs time vs validation valley location

### Exp 0 Paramters

%run -i 'arena.py'

# Parameters
weightsSource = '103' # trained on 3pc from scratch
# sourceNet = '103' # trained on 3pc from scratch
# sourceNet = '107' # trained on 4pc from scratch
# freeze = False
epochs = 350
expDescrBaseName = "Batch size test"

saveEveryRun = True # save stuff in results dir
saveWeightsCheckpoints = False # save chkp in results dit
saveTensorboardLogs = True # save logs in ./logs dir
# resID = '---NORESID---' # used when not saving data, but fitModel() still needs a resID

fractionOfDataToUse = 1
plotDuringTraining = False
loadWeights = False 
loadCheckpointWeights = False
askForConfirmation = False

### Create model and load data

# load data
X_train, X_test, y_train, y_test = loadData(randomState = None)



### Train

%run -i 'arena.py'

batches = [128, 1024]
for batch_size in batches:
    
    expDescr = expDescrBaseName + '_batchSize{}'.format(batch_size)
    
    # create model
    model, nnStr = createModel()

    # Prepare save dir
    if saveEveryRun:
        resID = genNextResultsDir(model)
        
    # train
    fitHistory, logDir = trainModel(resID, model, saveWeightsCheckpoints, saveTensorboardLogs)
        
    # score and save results
    score = calcScore(model)
    if saveEveryRun:
        saveTrainResults(resID, model, logDir, score)